# Couple candlestick - Analytics

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [10]:
# Triple candlesticks signal
data['three_up'] = (data['Close'] > data['Open']) & (data['Close'].shift(1) > data['Open'].shift(1)) & (data['Close'].shift(2) > data['Open'].shift(2))
data['three_down'] = (data['Close'] < data['Open']) & (data['Close'].shift(1) < data['Open'].shift(1)) & (data['Close'].shift(2) < data['Open'].shift(2))
data['has_signal'] = (data['three_up'] == True) | (data['three_down'] == True)

In [11]:
data[data['has_signal'] == True]

,Open,High,Low,Close,Volume,has_signal,three_up,three_down
Date,,,,,,,,
2020-11-02 09:45:00,898.8,898.8,897.7,898.1,2090,True,False,True
2020-11-02 10:10:00,899.9,899.9,898.9,899.7,1883,True,False,True
2020-11-02 13:10:00,899.2,900.4,898.9,899.3,1911,True,True,False
2020-11-02 14:05:00,901.5,901.7,897.0,897.9,3890,True,False,True
2020-11-02 14:10:00,898.0,898.5,896.6,897.7,7216,True,False,True
...,...,...,...,...,...,...,...,...
2025-02-14 11:05:00,1349.0,1349.2,1347.6,1347.8,3321,True,False,True
2025-02-14 11:10:00,1347.8,1347.9,1347.3,1347.5,1729,True,False,True
2025-02-14 11:15:00,1347.4,1347.5,1346.5,1346.8,2592,True,False,True


In [12]:
data['max_high_25bars_later'] = data['High'].shift(-25).rolling(25).max()
data['min_low_25bars_later'] = data['Low'].shift(-25).rolling(25).min()

In [15]:
def is_trap(r):
    trap = ''
    if r['has_signal'] == True:
        if r['three_up'] == True:
            # Long signal
            if r['min_low_25bars_later'] < r['Close'] - 3.5:
                trap = 'yes'
            else:
                trap = 'no'
        elif r['three_down'] == True:
            if r['max_high_25bars_later'] > r['Close'] + 3.5:
                trap = 'yes'
            else:
                trap = 'no'
    return trap

In [16]:
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [19]:
data[data.trap != ''].tail(15)

,Open,High,Low,Close,Volume,has_signal,three_up,three_down,max_high_25bars_later,min_low_25bars_later,trap
Date,,,,,,,,,,,
2025-02-13 10:20:00,1326.9,1327.5,1326.2,1326.4,2910,True,False,True,1338.5,1326.4,yes
2025-02-13 10:35:00,1327.4,1328.8,1327.4,1328.7,3455,True,True,False,1338.5,1328.6,no
2025-02-13 10:40:00,1328.7,1329.8,1328.6,1329.8,4716,True,True,False,1338.5,1329.4,no
2025-02-13 11:10:00,1333.3,1333.6,1332.5,1333.5,3764,True,True,False,1348.9,1332.5,no
2025-02-13 14:05:00,1335.6,1335.8,1334.4,1335.2,4360,True,False,True,1349.0,1333.4,yes
2025-02-13 14:10:00,1335.0,1335.2,1333.7,1333.8,6120,True,False,True,1349.0,1333.4,yes
2025-02-14 10:05:00,1346.5,1346.9,1346.1,1346.9,821,True,True,False,1350.4,1342.1,yes
2025-02-14 10:25:00,1346.1,1346.9,1346.0,1346.5,2143,True,True,False,1350.4,1341.1,yes
2025-02-14 10:30:00,1346.5,1347.6,1346.5,1347.1,3538,True,True,False,1350.4,1341.1,yes


In [20]:
len(data[data.trap != ''])

10047

In [21]:
len(data[data.has_signal == True])

10047